<a href="https://colab.research.google.com/github/CY-Chang-tw/Information-Retrieval-using-Pyserini/blob/main/IR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [7]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
import json
dic = {'id':None, 'contents':None}
articles = ''

with open('/content/input/wikidata_2021.json', 'w', encoding='utf-8') as wf:
  with open('/content/input/wiki_2021_08_05_1215639.json', 'r', encoding='utf-8') as rf:
    load_dict = json.load(rf)
    for d in load_dict:
      articles = d['title'] + d['articles']
      dic.update({'id': d['id'], 'contents':articles})
      wf.write(json.dumps(dic))
      wf.write('\n')

In [31]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input input \
  --language zh \
  --index indexes/wikidata_2021 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --bm25.accurate \
  --storePositions --storeDocvectors --storeRaw --storeContents --pretokenized --optimize --impact --keepStopwords

2023-03-16 11:57:38,087 INFO  [main] index.IndexCollection (IndexCollection.java:391) - Setting log level to INFO
2023-03-16 11:57:38,090 INFO  [main] index.IndexCollection (IndexCollection.java:394) - Starting indexer...
2023-03-16 11:57:38,090 INFO  [main] index.IndexCollection (IndexCollection.java:395) - ============ Loading Parameters ============
2023-03-16 11:57:38,090 INFO  [main] index.IndexCollection (IndexCollection.java:396) - DocumentCollection path: input
2023-03-16 11:57:38,091 INFO  [main] index.IndexCollection (IndexCollection.java:397) - CollectionClass: JsonCollection
2023-03-16 11:57:38,091 INFO  [main] index.IndexCollection (IndexCollection.java:398) - Generator: DefaultLuceneDocumentGenerator
2023-03-16 11:57:38,092 INFO  [main] index.IndexCollection (IndexCollection.java:399) - Threads: 1
2023-03-16 11:57:38,092 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Language: zh
2023-03-16 11:57:38,093 INFO  [main] index.IndexCollection (IndexCollection.

In [44]:
!python -m pyserini.search.lucene \
  --index indexes/wikidata_2021 \
  --topics bm25/wiki.tsv \
  --output run.wiki_zh.txt \
  --language zh \
  --bm25 \
  --hits 100

Running bm25/wiki.tsv topics, saving to run.wiki_zh.txt...
100% 1/1 [00:00<00:00,  3.12it/s]


In [45]:
! cat run.wiki_zh.txt

1 Q0 488525 1 14.446400 Anserini
1 Q0 1137603 2 14.415000 Anserini
1 Q0 245815 3 14.402000 Anserini
1 Q0 247385 4 14.397900 Anserini
1 Q0 1199669 5 14.393000 Anserini
1 Q0 647348 6 14.378600 Anserini
1 Q0 587367 7 14.375200 Anserini
1 Q0 289862 8 14.370900 Anserini
1 Q0 815791 9 14.367700 Anserini
1 Q0 1137420 10 14.350400 Anserini
1 Q0 1167598 11 14.349200 Anserini
1 Q0 660215 12 14.346500 Anserini
1 Q0 71183 13 14.345700 Anserini
1 Q0 941587 14 14.323700 Anserini
1 Q0 918118 15 14.323200 Anserini
1 Q0 298377 16 14.316900 Anserini
1 Q0 199211 17 14.313000 Anserini
1 Q0 475839 18 14.301200 Anserini
1 Q0 417600 19 14.293800 Anserini
1 Q0 640617 20 14.284600 Anserini
1 Q0 103910 21 14.281300 Anserini
1 Q0 1208679 22 14.265200 Anserini
1 Q0 1206923 23 14.261600 Anserini
1 Q0 752899 24 14.259100 Anserini
1 Q0 290470 25 14.257100 Anserini
1 Q0 249910 26 14.247300 Anserini
1 Q0 113906 27 14.247200 Anserini
1 Q0 239924 28 14.245000 Anserini
1 Q0 710072 29 14.238400 Anserini
1 Q0 107075 30 14.

In [80]:
import time
from pyserini.search.lucene import LuceneSearcher
from pyserini.index.lucene import IndexReader
import itertools

searcher = LuceneSearcher('indexes/wikidata_2021')
time_start = time.time() #開始計時
hits = searcher.search('蝙蝠俠')
time_end = time.time()    #結束計時

time_inverted_index = time_end - time_start   #執行所花時間
print('time cost', time_inverted_index, 's')

print('hits: ')
for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print(hits[i].raw)
print()



# Initialize from an index path:
index_reader = IndexReader('indexes/wikidata_2021')


# access basic index statistics
print(f'basic index statistics: "{index_reader.stats()}"')
print('-----------------------------------------------------------------------')

for term in itertools.islice(index_reader.terms(), 10):
    print(f'{term.term} (df={term.df}, cf={term.cf})')

term = '蝙蝠俠'
# Analyze the term.
analyzed = index_reader.analyze(term)
print(f'The analyzed form of "{term}" is "{analyzed[0]}"')

df, cf = index_reader.get_term_counts(analyzed[0], analyzer=None)
print(f'term "{term}": df={df}, cf={cf}')



# Fetch and traverse postings for an analyzed term:
postings_list = index_reader.get_postings_list(analyzed[0])
print('Posting: ', end='')
for posting in postings_list:
    print(f'docid={posting.docid}, tf={posting.tf}, pos={posting.positions}')
print()


time cost 0.005647897720336914 s
hits: 
 1 115501 11.28800
{
  "id" : 115501,
  "contents" : "瑞芳蝙蝠洞瑞芳蝙蝠洞，是位於臺灣新北市瑞芳區海濱裏北部濱海公路旁的摺翅蝠棲息地。此洞位在臺2線75K處。昔日瑞芳多天然山區，因林相完整、蚊蠅多，吸引摺翅蝠在此大量聚集，將此洞作生殖育幼用的繁殖洞，為東亞最多摺翅蝠之處。有遷移度冬習性的摺翅蝠，從四月起母蝠起進駐洞內，於七、八月生出幼蝠，九月以後母蝠帶著幼蝠飛離此洞，部分可能遷至臺灣南部過冬。如一處在旗山山區渠道的摺翅蝠冬眠洞，學會選擇不公佈地點，以防幹擾。絕大多數仍不確定度冬的地點。西濱公路完工前即有蝙蝠，當時卻沒有正式研究，數字不可考；西濱公路完工通車，夏季傍晚蝙蝠出巢的壯觀景象吸引無數觀蝠人潮，因而瑞芳蝙蝠洞開始成為著名景點。然好景不常，而約末在80年代據當地民眾表示有人拿沖天炮往蝙蝠洞內炸，蝙蝠一夕之間全數離開，空洞時間達十餘年年之後數量又緩慢恢復。2005年，瑞芳鎮公所委託臺灣蝙蝠學會調查，蝙蝠數量高達約五十萬隻，但2007年數量再次遞減，只剩最盛期一半。2014年，調查約廿萬隻左右。2010年開始，臺北市蝙蝠保育學會與瑞芳區公所進行合作，每年夏季辦理蝙蝠活動，藉由賞蝠活動推動蝙蝠保育。其數量驟減原因包括氣候變遷、棲息地破壞、草地農藥施肥、員山子分洪道炸山讓蝙蝠洞內疑有大面積坍塌，造成棲息空間縮小。除疾駛車輛造成幼小的褶翅蝠被氣流打落死亡外，還有被瓜棚阻攔飛行路線，是因為外地民眾在洞口前開闢成菜園，更被亂丟垃圾。甚至有電信業者在這裡架設基地臺2019年由公路局總局基隆公務段與臺北市蝙蝠保育學會合作，將臺2線7傍晚6至8時5K降段於蝙蝠棲息育幼時進行限速，希冀檢少蝙蝠傷亡。新北市政府民政局和臺電、中油等單位合作，修整蝙蝠洞公園、增建賞蝠平臺，吸引民眾到訪，當地不僅賞蝠視野佳，更是觀海休憩地點。政府和當地海濱裏裏民合作保育和觀光，除設立小心蝙蝠的提醒駕駛人告示外，沿途有蝙蝠壁畫，鐵路也變身為蝙蝠橋，希望藉此複製猴硐經驗，帶動此地觀光。公園採全天開放，內設停車場與涼亭，附近有海濱車站遺跡。2015年報導時，瑞芳區公所於蝙蝠活躍季節的七月份會在每星期六、日下午五點半時至七點半時，也就是蝙蝠傍晚出洞覓食之時，定點定時排定志工導覽解說，